In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

## 1. Imports and Model Loading

In [1]:
import os
import imageio
import uuid
from IPython.display import Image as ImageDisplay
from inference import Inference, ready_gaussian_for_video_rendering, render_video, load_image, load_mask, load_single_mask, display_image, make_scene, interactive_visualizer

/home/leelab/anaconda3/envs/sam3d-objects/lib/python3.11/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Warp 1.10.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 4090" (24 GiB, sm_89, mempool enabled)
     "cuda:1"   : "NVIDIA GeForce RTX 4090" (24 GiB, sm_89, mempool enabled)
   CUDA peer access:
     Not supported
   Kernel cache:
     /home/leelab/.cache/warp/1.10.0


2025-11-25 05:59:23.250 | INFO     | sam3d_objects.pipeline.inference_pipeline:set_attention_backend:15 - GPU name is NVIDIA GeForce RTX 4090


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2025-11-25 05:59:24.406 | INFO     | sam3d_objects.model.backbone.tdfy_dit.modules.sparse:__from_env:39 - [SPARSE] Backend: spconv, Attention: sdpa
2025-11-25 05:59:26.688 | INFO     | sam3d_objects.model.backbone.tdfy_dit.modules.attention:__from_env:30 - [ATTENTION] Using backend: sdpa


[SPARSE][CONV] spconv algo: auto


2025-11-25 05:59:27.109 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
2025-11-25 05:59:27.110 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 


In [2]:
PATH = os.getcwd()
TAG = "hf"
config_path = f"{PATH}/../checkpoints/{TAG}/pipeline.yaml"
inference = Inference(config_path, compile=False)

2025-11-25 05:59:27.131 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
/home/leelab/anaconda3/envs/sam3d-objects/lib/python3.11/site-packages/moge/model/v1.py:171: UserWarning: The following deprecated/invalid arguments are ignored: {'output_mask': True, 'split_head': True}
  warnings.warn(f"The following deprecated/invalid arguments are ignored: {deprecated_kwargs}")
2025-11-25 05:59:31.087 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
2025-11-25 05:59:31.088 | INFO     | sam3d_objects.pipeline.inference_pipeline:__init__:98 - self.device: cuda
2025-11-25 05:59:31.089 | INFO     | sam3d_objects.pipeline.inference_pipeline:__init__:99 - CUDA_VISIBLE_DEVICES: None
2025-11-25 05:59:31.089 | INFO     | sam3d_objects.pipeline.inference_pipeline:__init__:100 - Actually using GPU: 0
2025-11-25 05:59:31.089 | INFO     

## 2. Load input image to lift to 3D (single object)

In [ ]:
IMAGE_PATH = f"{PATH}/images/shutterstock_stylish_kidsroom_1640806567/image.png"
IMAGE_NAME = os.path.basename(os.path.dirname(IMAGE_PATH))

image = load_image(IMAGE_PATH)
mask = load_single_mask(os.path.dirname(IMAGE_PATH), index=14)
display_image(image, masks=[mask])

## 3. Generate Gaussian Splat (single view)

In [ ]:
# run model
output = inference(image, mask, seed=42)

# export gaussian splat (as point cloud)
output["gs"].save_ply(f"{PATH}/gaussians/single/{IMAGE_NAME}.ply")

## 4. (Optional) Multi-view conditioning from an auxiliary directory
Provide extra camera views in a directory and pass it as the third argument to `inference(image, mask, aux_views_dir)`.
The snippet below fabricates a tiny auxiliary set so you can see the call pattern; replace `AUX_VIEWS_DIR` with your own folder of RGBA/RGB images.


In [3]:
from pathlib import Path
from PIL import Image
import numpy as np

IMG_DIR = '/home/leelab/sam3dnate/testImgs'
image = load_image(os.path.join(IMG_DIR, 'G1_run1_mouse-frame_synced_frame2230.jpg'))
mask = load_mask(os.path.join(IMG_DIR, 'G1_run1_mouse-frame_synced_frame2230.png'))

AUX_VIEWS_DIR = Path(f"{IMG_DIR}/aux")
#AUX_VIEWS_DIR.mkdir(parents=True, exist_ok=True)

MULTI_VIEW_NAME = AUX_VIEWS_DIR.name
output_multiview = inference(image, mask, aux_views=str(AUX_VIEWS_DIR), seed=7)
os.makedirs(f"{PATH}/gaussians/multiview", exist_ok=True)
output_multiview["gs"].save_ply(f"{PATH}/gaussians/multiview/{MULTI_VIEW_NAME}.ply")


TypeError: InferencePipelinePointMap.run() got an unexpected keyword argument 'aux_views'

In [ ]:
# Choose which reconstruction to visualize (defaults to single-view output)
visualization_output = output_multiview if 'output_multiview' in globals() else output
visualization_name = MULTI_VIEW_NAME if 'output_multiview' in globals() else IMAGE_NAME
visualization_dir = 'multiview' if 'output_multiview' in globals() else 'single'
os.makedirs(f"{PATH}/gaussians/{visualization_dir}", exist_ok=True)


## 5. Visualize Gaussian Splat

In [ ]:
# render gaussian splat
scene_gs = make_scene(visualization_output)
scene_gs = ready_gaussian_for_video_rendering(scene_gs)

video = render_video(
    scene_gs,
    r=1,
    fov=60,
    pitch_deg=15,
    yaw_start_deg=-45,
    resolution=512,
)["color"]

# save video as gif
imageio.mimsave(
    os.path.join(f"{PATH}/gaussians/{visualization_dir}/{visualization_name}.gif"),
    video,
    format="GIF",
    duration=1000 / 30,  # default assuming 30fps from the input MP4
    loop=0,  # 0 means loop indefinitely
)

# notebook display
ImageDisplay(url=f"gaussians/{visualization_dir}/{visualization_name}.gif?cache_invalidator={uuid.uuid4()}")


### b. Interactive Visualizer

In [ ]:
# might take a while to load (black screen)
interactive_visualizer(f"{PATH}/gaussians/{visualization_dir}/{visualization_name}.ply")
